In [2]:
import pandas as pd



In [3]:
tokens = pd.read_csv (r'Traits.csv')
sales = pd.read_csv (r'Sales.csv')

In [4]:
tokens

,Unnamed: 0,collection,contract,token_id,nft_url,trait_type,value
0,0,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Type,Human
1,1,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Hair,Water
2,2,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Clothing,Pink Oversized Kimono
3,3,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Eyes,Striking
4,4,azuki,0xed5af388653567af2f388e6224dc7c4b3241c544,0,https://ikzttp.mypinata.cloud/ipfs/QmQFkLSQysj...,Mouth,Frown
...,...,...,...,...,...,...,...
696003,44435,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Background,Blue
696004,44436,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Skin,Olive Green
696005,44437,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Body,Puffer Green
696006,44438,pudgy_penguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,8887,https://ipfs.io/ipfs/QmWXJXRdExse2YHRY21Wvh4pj...,Face,Handlebar Bear


In [5]:
print(pd.unique(tokens['token_id']))
print(tokens.groupby('collection')['token_id'].nunique())

[    0     1     2 ... 19997 19999 20000]
collection
BAYC              10000
MAYC              18502
azuki             10000
clone_x           18895
cool_cats          9933
doodles           10000
meebits           20000
pudgy_penguins     8888
Name: token_id, dtype: int64


In [6]:
grouped = tokens.groupby('collection')
bayc = grouped.get_group('BAYC')
mayc = grouped.get_group('MAYC')
azuki = grouped.get_group('azuki')
clone_x = grouped.get_group('clone_x')
cool_cats = grouped.get_group('cool_cats')
doodles = grouped.get_group('doodles')
meebits = grouped.get_group('meebits')
pudgy_penguins = grouped.get_group('pudgy_penguins')

In [7]:
bayc
print(bayc.groupby('trait_type')['token_id'].nunique())
print(bayc.groupby('trait_type')['value'].nunique())
print(pd.unique(bayc.groupby('trait_type').get_group('Background')['value']))
print(bayc.groupby('trait_type').get_group('Background')['value'].value_counts()/10000)

trait_type
Background    10000
Clothes        8114
Earring        2977
Eyes          10000
Fur           10000
Hat            7744
Mouth         10000
Name: token_id, dtype: int64
trait_type
Background     8
Clothes       43
Earring        6
Eyes          23
Fur           19
Hat           36
Mouth         33
Name: value, dtype: int64
['Orange' 'Aquamarine' 'Purple' 'Blue' 'Army Green' 'Yellow' 'Gray'
 'New Punk Blue']
Purple           0.1291
Yellow           0.1283
Orange           0.1273
Aquamarine       0.1266
Army Green       0.1243
Blue             0.1242
New Punk Blue    0.1232
Gray             0.1170
Name: value, dtype: float64


In [8]:
sales[sales['tokenID'] == 0]

,blockNumber,timeStamp,hash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,gasPrice,currency,amount,amountUsd
0,13988541,1641961048,0xc208fdb2f133bda64522fececd6518a565aaa6e8801b...,0x0000000000000000000000000000000000000000,0xed5af388653567af2f388e6224dc7c4b3241c544,0xd45058bf25bbd8f586124c479d384c8c708ce23a,0,Azuki,AZUKI,144500135604,ETH,0.000000,0.000000
41274,12292922,1619133220,0xcfb197f62ec5c7f0e71a11ec0c4a0e394a3aa41db538...,0x0000000000000000000000000000000000000000,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,0,BoredApeYachtClub,BAYC,88000000000,ETH,0.000000,0.000000
41760,12344148,1619817106,0x8a65c8db391825872ba66a06352cc62ac414ceb6e37b...,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xf7801b8115f3fe46ac55f8c0fdb5243726bdb66a,0,BoredApeYachtClub,BAYC,46000000000,ETH,0.000000,0.000000
104449,13117215,1630197658,0x612a5190e324f5455432af3aa2c97e9d35f036929263...,0x0000000000000000000000000000000000000000,0x60e4d786628fea6478f785a6d7e704777c86a7c6,0x9056d15c49b19df52ffad1e6c11627f035c0c960,0,MutantApeYachtClub,MAYC,51851145919,ETH,2.999444,9779.687491
296448,13449414,1634663005,0x663e8b5d7bd8871fba2b1fc969de039c0169e6e29222...,0xeff53613068a15a6845943e447fca03936c283c8,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,0xad097fdcd58535250c59807d6683e0a6b688d6cc,0,Doodles,DOODLE,111000000000,ETH,10.000000,38090.531279
326926,14312286,1646287360,0x05cf0366a09c22cd845dcf673a99bb8d7ecdfceb796e...,0xad097fdcd58535250c59807d6683e0a6b688d6cc,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,0x620b70123fb810f6c653da7644b5dd0b6312e4d8,0,Doodles,DOODLE,47742340029,ETH,0.000000,0.000000
344365,12755983,1625333674,0x9b4c1a1e0e72d41b85c2fec185433f95b63fa27178c8...,0xfc86a64a8de22cf25410f7601acbd8d6630da93d,0x1a92f7381b9f03921564a437210bb9396471050c,0x98bb3a3921200017ebd0af803aeddd464e70e791,0,Cool Cats,COOL,13000000000,ETH,0.000000,0.000000
378259,13911124,1640926643,0x6fc306f30007dc1c78141c27a7a8dbe7919814866c90...,0x98bb3a3921200017ebd0af803aeddd464e70e791,0x1a92f7381b9f03921564a437210bb9396471050c,0xd9463b55af194d4fa22a532a52f5bdd45b66d290,0,Cool Cats,COOL,113793925785,ETH,0.000000,0.000000


In [9]:
print(sales.groupby('tokenName')['tokenID'].nunique())

tokenName
Azuki                 10000
BoredApeYachtClub     10000
CloneX                18554
Cool Cats              9938
Doodles                9784
Meebits               19999
MutantApeYachtClub    17990
PudgyPenguins          8887
Name: tokenID, dtype: int64


In [10]:
# pd.get_dummies(bayc['value'])
bayc_features = bayc.groupby('token_id')#.filter('trait_type')
print(bayc_features)

In [13]:
df = pd.DataFrame({bayc['trait_type'] : bayc['value']}, index=bayc['token_id'])
print(df)

TypeError: unhashable type: 'Series'

In [ ]:
bayc_nft = []
for id in bayc['token_id'].unique():
    for _, row in bayc[bayc.token_id == id].itterrows():
        row.trait_type : row.value